# BERT Question AND Answering System

  Siddhesh Dwarkanath Munagekar : W0747944



#Importing necessary Libraries

In [23]:
!pip install transformers

Ref :This is a modified version of the original work  by ChrisMcKormic.

In [24]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


In [25]:
question = "How many parameters does BERT-large have?"
answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

input_ids = tokenizer(question, answer_text, return_tensors='pt')
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])
outputs = model(**input_ids, start_positions=start_positions, end_positions=end_positions)
print(outputs)
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits)
print(answer_start)
print(answer_end)

QuestionAnsweringModelOutput(loss=tensor(12.5008, grad_fn=<DivBackward0>), start_logits=tensor([[-6.4849, -6.4358, -8.1077, -8.8489, -7.8751, -8.0522, -8.4684, -8.5295,
         -7.7074, -9.2464, -6.4849, -2.7303, -6.3473, -5.7299, -7.7780, -7.0391,
         -6.3331, -7.3153, -7.3048, -7.4121, -2.2534, -5.3971, -0.9424, -7.3584,
         -5.4575, -7.0769, -4.4887, -3.9272, -5.6967, -5.9506, -5.0059, -5.9812,
          0.0530, -5.5968, -4.7093, -4.5750, -6.1786, -2.2294, -0.1904, -0.2327,
         -2.7331,  6.4256, -2.6543, -4.5655, -4.9872, -4.9834, -5.9110, -7.8402,
         -1.8986, -7.2123, -4.1543, -6.2354, -8.0953, -7.2329, -6.4411, -6.8384,
         -8.1032, -7.0570, -7.7332, -6.8711, -7.1045, -8.2966, -6.1939, -8.0817,
         -7.5501, -5.9695, -8.1008, -6.8849, -8.2273, -6.4850]],
       grad_fn=<SqueezeBackward1>), end_logits=tensor([[-2.0629, -6.3878, -6.2450, -6.3605, -7.0722, -7.6281, -7.1160, -6.8674,
         -7.1313, -7.1495, -2.0628, -5.0858, -4.7276, -3.5955, -6.3050,

In [26]:
outputs = model(**input_ids, start_positions=start_positions, end_positions=end_positions)

In [27]:
input_ids = tokenizer.encode(question, answer_text)
tokens =tokenizer.convert_ids_to_tokens(input_ids)

sep_index = input_ids.index(tokenizer.sep_token_id)
num_seg_a = sep_index + 1
num_seg_b = len(input_ids) - num_seg_a
segment_ids = [0]*num_seg_a + [1]*num_seg_b
assert len(segment_ids) == len(input_ids)

outputs = model(torch.tensor([input_ids]),token_type_ids=torch.tensor([segment_ids]),
                start_positions=start_positions, end_positions=end_positions)
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits)
print(answer_start)
print(answer_end)
tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # Start with the first token.
answer = tokens[answer_start]
    # Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
          # If it's a subword token, then recombine it with the previous token.
  if tokens[i][0:2] == '##':

    answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
  else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')
    
        


tensor(41)
tensor(42)
Answer: "340m"


# Function Creation to deploy it on Front End

In [28]:
def question_answer(question, answer_text):
    input_ids = tokenizer.encode(question, answer_text)
    tokens =tokenizer.convert_ids_to_tokens(input_ids)

    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)

    outputs = model(torch.tensor([input_ids]),token_type_ids=torch.tensor([segment_ids]),
                start_positions=start_positions, end_positions=end_positions)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)
    print(answer_start)
    print(answer_end)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # Start with the first token.
    answer = tokens[answer_start]
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):

      # If it's a subword token, then recombine it with the previous token.
      if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
        # Otherwise, add a space then the token.
      else:
        answer += ' ' + tokens[i]

            

    print('Answer: "' + answer + '"')
    return answer

# Text Wraper to wrap up the text

In [29]:
import textwrap

wrapper=textwrap.TextWrapper(width=80)
bert_abstract = "The answer is not in this sentence.The immune system is a system of many biological structures and processes within an organism that protects against disease. To function properly, an immune system must detect a wide variety of agents, known as pathogens, from viruses to parasitic worms, and distinguish them from the organism's own healthy tissue. In many species, the immune system can be classified into subsystems, such as the innate immune system versus the adaptive immune system, or humoral immunity versus cell-mediated immunity. In humans, the blood–brain barrier, blood–cerebrospinal fluid barrier, and similar fluid–brain barriers separate the peripheral immune system from the neuroimmune system which protects the brain."
bert_abstract

"The answer is not in this sentence.The immune system is a system of many biological structures and processes within an organism that protects against disease. To function properly, an immune system must detect a wide variety of agents, known as pathogens, from viruses to parasitic worms, and distinguish them from the organism's own healthy tissue. In many species, the immune system can be classified into subsystems, such as the innate immune system versus the adaptive immune system, or humoral immunity versus cell-mediated immunity. In humans, the blood–brain barrier, blood–cerebrospinal fluid barrier, and similar fluid–brain barriers separate the peripheral immune system from the neuroimmune system which protects the brain."

In [30]:
question="What does the immune system protect against?"
question_answer(question,bert_abstract)

tensor(36)
tensor(36)
Answer: "disease"


'disease'

In [31]:
!pip install jupyter-dash
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State# Load Data
import dash

#Code for Front End Web App.....

In [32]:
###Code for Front End Web App.....
df = px.data.tips()# Build App
app = JupyterDash(__name__)

app.layout = html.Div([
             html.H1("Question Answering System"),
             html.Label('How can I help You Today'),
             
             dcc.Input(id='question',
                 placeholder='Please enter your Question here',
                 type='text',
                 value='',
                 autoComplete='on'
             ),
             html.Button('Ask',id='submit-val',n_clicks=0),
             html.Div(id='container-button-basic',children='Enter a value and press submit')
    
                      ])
@app.callback(
    dash.dependencies.Output('container-button-basic', 'children'),
    [dash.dependencies.Input('submit-val', 'n_clicks')],
    [dash.dependencies.State('question', 'value')])
def update_output(n_clicks, value):
  if n_clicks !=0:
    
    return "Answer :",question_answer(value,bert_abstract)

Developed by Siddhesh D. Munagekar
W0747944

# External web link to see the output on front end

In [33]:
app.run_server(mode='external')

Dash app running on:


<IPython.core.display.Javascript object>

# Example Questions to try on front end
- What does the immune system protect against?
- What is the subsystem that protects the human brain?
- What are two of its subsystems?
- What are the agents detected by the immune system called?